In [1]:
import os
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"

In [2]:
from f1tenth_gym_jax import make
from f1tenth_gym_jax.envs.utils import Param
import jax
import numpy as np

In [3]:
num_agents = 5
num_envs = 100

In [4]:
env = make(f"Spielberg_{num_agents}_noscan_time_v0")

2024-11-12 11:04:31.589242: W external/xla/xla/service/gpu/nvptx_compiler.cc:930] The NVIDIA driver's CUDA version is 12.2 which is older than the PTX compiler version 12.6.77. Because the driver is older than the PTX compiler version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


In [5]:
rng = jax.random.PRNGKey(0)
rng, _rng = jax.random.split(rng)
reset_rng = jax.random.split(_rng, num_envs)

In [6]:
obsv, env_state = jax.vmap(env.reset)(reset_rng)

/home/billyzheng/f1tenth_gym_jax/venv/lib/python3.10/site-packages/jax/_src/numpy/lax_numpy.py:6217: UserWarning: Explicitly requested dtype float64 requested in asarray is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/jax-ml/jax#current-gotchas for more.
  start = asarray(start, dtype=computation_dtype)
/home/billyzheng/f1tenth_gym_jax/venv/lib/python3.10/site-packages/jax/_src/numpy/lax_numpy.py:6218: UserWarning: Explicitly requested dtype float64 requested in asarray is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/jax-ml/jax#current-gotchas for more.
  stop = asarray(stop, dtype=computation_dtype)
/home/billyzheng/f1tenth_gym_jax/venv/lib/python3.10/site-packages/jax/_src/numpy/lax_numpy.

In [7]:
rng, _rng = jax.random.split(rng)
action_rng = jax.random.split(_rng, num_envs*num_agents).reshape((num_agents, num_envs, -1))
actions = {k: jax.vmap(env.action_space(k).sample)(action_rng[i]) for i, k in enumerate(env.agents)}

In [8]:
rng, _rng = jax.random.split(rng)
step_rng = jax.random.split(_rng, num_envs)
obsv, env_state, _, _, info = jax.vmap(env.step)(step_rng, env_state, actions)

In [9]:
def env_init(rng):
    rng, _rng = jax.random.split(rng)
    reset_rng = jax.random.split(_rng, num_envs)
    obsv, env_state = jax.vmap(env.reset)(reset_rng)
    return (env_state, obsv, rng)

def env_step(runner_state, unused):
    env_state, last_obs, rng = runner_state

    rng, _rng = jax.random.split(rng)
    action_rng = jax.random.split(_rng, num_envs*num_agents).reshape((num_agents, num_envs, -1))
    actions = {k: jax.vmap(env.action_space(k).sample)(action_rng[i]) for i, k in enumerate(env.agents)}
    for k in env.agents:
        actions[k] = actions[k].at[:, 0].set(0.0)
        actions[k] = actions[k].at[:, 1].set(2.0)

    rng, _rng = jax.random.split(rng)
    step_rng = jax.random.split(_rng, num_envs)
    obsv, env_state, _, _, info = jax.vmap(env.step)(step_rng, env_state, actions)

    runner_state = (env_state, obsv, rng)
    return runner_state, runner_state

In [15]:
runner_state = env_init(rng)
last_runner_state, all_runner_states = jax.lax.scan(env_step, runner_state, length=1000)

In [16]:
all_runner_states[0].cartesian_states.shape

(1000, 100, 5, 7)

In [17]:
import importlib
# import f1tenth_gym_jax
import f1tenth_gym_jax.envs.rendering.renderer as r
importlib.reload(r)

<module 'f1tenth_gym_jax.envs.rendering.renderer' from '/home/billyzheng/f1tenth_gym_jax/f1tenth_gym_jax/envs/rendering/renderer.py'>

In [18]:
player = r.TrajRenderer(env)

In [19]:
try:
    player.render(np.array(all_runner_states[0].cartesian_states))
except Exception as e:
    print(e)
    player.close()

/home/billyzheng/f1tenth_gym_jax/venv/lib/python3.10/site-packages/pyqtgraph/graphicsItems/ViewBox/ViewBox.py:641: RuntimeWarning: overflow encountered in cast
  if maxRng[ax] is not None and diff > maxRng[ax]:
/home/billyzheng/f1tenth_gym_jax/venv/lib/python3.10/site-packages/pyqtgraph/graphicsItems/ViewBox/ViewBox.py:654: RuntimeWarning: overflow encountered in cast
  if lmn is not None and mn < lmn:
/home/billyzheng/f1tenth_gym_jax/venv/lib/python3.10/site-packages/pyqtgraph/graphicsItems/ViewBox/ViewBox.py:658: RuntimeWarning: overflow encountered in cast
  elif lmx is not None and mx > lmx:
/home/billyzheng/f1tenth_gym_jax/venv/lib/python3.10/site-packages/pyqtgraph/graphicsItems/ViewBox/ViewBox.py:1640: RuntimeWarning: overflow encountered in cast
  if maxRng[axis] is not None and maxRng[axis] < range:
/home/billyzheng/f1tenth_gym_jax/venv/lib/python3.10/site-packages/pyqtgraph/graphicsItems/ViewBox/ViewBox.py:1643: RuntimeWarning: overflow encountered in cast
  if limits[axis][0

rewind to 521
rewind to 470
rewind to 391
rewind to 307
rewind to 223
rewind to 139
fast forward to step 318
fast forward to step 431
fast forward to step 542
fast forward to step 654
fast forward to step 765
fast forward to step 883
play pause
play pause


/home/billyzheng/f1tenth_gym_jax/venv/lib/python3.10/site-packages/pyqtgraph/graphicsItems/ViewBox/ViewBox.py:641: RuntimeWarning: overflow encountered in cast
  if maxRng[ax] is not None and diff > maxRng[ax]:
/home/billyzheng/f1tenth_gym_jax/venv/lib/python3.10/site-packages/pyqtgraph/graphicsItems/ViewBox/ViewBox.py:654: RuntimeWarning: overflow encountered in cast
  if lmn is not None and mn < lmn:
/home/billyzheng/f1tenth_gym_jax/venv/lib/python3.10/site-packages/pyqtgraph/graphicsItems/ViewBox/ViewBox.py:658: RuntimeWarning: overflow encountered in cast
  elif lmx is not None and mx > lmx:
/home/billyzheng/f1tenth_gym_jax/venv/lib/python3.10/site-packages/pyqtgraph/graphicsItems/ViewBox/ViewBox.py:1640: RuntimeWarning: overflow encountered in cast
  if maxRng[axis] is not None and maxRng[axis] < range:
/home/billyzheng/f1tenth_gym_jax/venv/lib/python3.10/site-packages/pyqtgraph/graphicsItems/ViewBox/ViewBox.py:1643: RuntimeWarning: overflow encountered in cast
  if limits[axis][0

rewind to 853


/home/billyzheng/f1tenth_gym_jax/venv/lib/python3.10/site-packages/pyqtgraph/graphicsItems/ViewBox/ViewBox.py:641: RuntimeWarning: overflow encountered in cast
  if maxRng[ax] is not None and diff > maxRng[ax]:
/home/billyzheng/f1tenth_gym_jax/venv/lib/python3.10/site-packages/pyqtgraph/graphicsItems/ViewBox/ViewBox.py:654: RuntimeWarning: overflow encountered in cast
  if lmn is not None and mn < lmn:
/home/billyzheng/f1tenth_gym_jax/venv/lib/python3.10/site-packages/pyqtgraph/graphicsItems/ViewBox/ViewBox.py:658: RuntimeWarning: overflow encountered in cast
  elif lmx is not None and mx > lmx:
/home/billyzheng/f1tenth_gym_jax/venv/lib/python3.10/site-packages/pyqtgraph/graphicsItems/ViewBox/ViewBox.py:1640: RuntimeWarning: overflow encountered in cast
  if maxRng[axis] is not None and maxRng[axis] < range:
/home/billyzheng/f1tenth_gym_jax/venv/lib/python3.10/site-packages/pyqtgraph/graphicsItems/ViewBox/ViewBox.py:1643: RuntimeWarning: overflow encountered in cast
  if limits[axis][0

...

rewind to 300


/home/billyzheng/f1tenth_gym_jax/venv/lib/python3.10/site-packages/pyqtgraph/graphicsItems/ViewBox/ViewBox.py:641: RuntimeWarning: overflow encountered in cast
  if maxRng[ax] is not None and diff > maxRng[ax]:
/home/billyzheng/f1tenth_gym_jax/venv/lib/python3.10/site-packages/pyqtgraph/graphicsItems/ViewBox/ViewBox.py:654: RuntimeWarning: overflow encountered in cast
  if lmn is not None and mn < lmn:
/home/billyzheng/f1tenth_gym_jax/venv/lib/python3.10/site-packages/pyqtgraph/graphicsItems/ViewBox/ViewBox.py:658: RuntimeWarning: overflow encountered in cast
  elif lmx is not None and mx > lmx:
/home/billyzheng/f1tenth_gym_jax/venv/lib/python3.10/site-packages/pyqtgraph/graphicsItems/ViewBox/ViewBox.py:1640: RuntimeWarning: overflow encountered in cast
  if maxRng[axis] is not None and maxRng[axis] < range:
/home/billyzheng/f1tenth_gym_jax/venv/lib/python3.10/site-packages/pyqtgraph/graphicsItems/ViewBox/ViewBox.py:1643: RuntimeWarning: overflow encountered in cast
  if limits[axis][0

rewind to 211


/home/billyzheng/f1tenth_gym_jax/venv/lib/python3.10/site-packages/pyqtgraph/graphicsItems/ViewBox/ViewBox.py:641: RuntimeWarning: overflow encountered in cast
  if maxRng[ax] is not None and diff > maxRng[ax]:
/home/billyzheng/f1tenth_gym_jax/venv/lib/python3.10/site-packages/pyqtgraph/graphicsItems/ViewBox/ViewBox.py:654: RuntimeWarning: overflow encountered in cast
  if lmn is not None and mn < lmn:
/home/billyzheng/f1tenth_gym_jax/venv/lib/python3.10/site-packages/pyqtgraph/graphicsItems/ViewBox/ViewBox.py:658: RuntimeWarning: overflow encountered in cast
  elif lmx is not None and mx > lmx:
/home/billyzheng/f1tenth_gym_jax/venv/lib/python3.10/site-packages/pyqtgraph/graphicsItems/ViewBox/ViewBox.py:1640: RuntimeWarning: overflow encountered in cast
  if maxRng[axis] is not None and maxRng[axis] < range:
/home/billyzheng/f1tenth_gym_jax/venv/lib/python3.10/site-packages/pyqtgraph/graphicsItems/ViewBox/ViewBox.py:1643: RuntimeWarning: overflow encountered in cast
  if limits[axis][0

play pause
play pause
play pause
